In [1]:
# imports
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import itertools
import math 
import statistics
from itertools import permutations
import json
import time
import joblib

# from google.colab import drive
# drive.mount('/content/gdrive/', force_remount=True)

In [2]:
# cd '/content/gdrive/MyDrive/Colab Notebooks/vis-2-assignment'

In [3]:
cars = pd.read_csv("data/cars/cars.csv", index_col=0)

In [4]:
cars = cars[cars.horsepower != "?"]

## FS1

In [5]:
def isfloat(num):
    try:
        float(num)
        return True
    except ValueError:
        return False

def calculate_FS1_for_each_column(df,group_label):
    
    col = df.columns
    colu = []
    for c in col:
        if (isfloat(df[c][0])):
            colu.append(c)
            
    for c in colu:
        df[c] = df[c].astype(float)
    
    string_columns_float = list(df.select_dtypes(exclude=['object']).columns)
    string_columns_object = list(df.select_dtypes(include=['object']).columns)
    
    # print(f"Group label is: {group_label}")
    # print(f"Group label has {len(df[group_label].unique())} unique values")
    no_of_datasets = len(df[group_label].unique())
    df = df.drop(labels=string_columns_object,axis=1)
    list_of_datasets = []
    origin_dict = {}
    dict_of_mean_values = {} #key is the origin and the values are list of means for each column of a datas

    #creating separated datasetst for each group
    for count,value in enumerate(df[group_label].unique()):
        list_of_datasets.append( df[df[group_label] == value])
        list_of_datasets[count] = list_of_datasets[count].reset_index()
    
    
    #drop origin column for each group and save it as a key in dictionary where 
    #the value of the key will be the corresponding dataset
    
    for dataset in list_of_datasets:
        
        group = int(dataset[group_label].unique()[0])
        
        origin_dict[group] = dataset.drop(labels=[group_label],axis=1)
        
        columns = origin_dict[group].columns
        scaler = MinMaxScaler()
        scaler.fit(origin_dict[group])
        origin_dict[group] = scaler.transform(origin_dict[group])
        origin_dict[group] = pd.DataFrame(origin_dict[group], columns = columns)
        origin_dict[group] = origin_dict[group].drop(labels=["index"] ,axis=1)
        dict_of_mean_values[group] = [origin_dict[group][c].mean() for c in origin_dict[group].columns]
        
    #calculation of diff and QDS between each combination of the groups.
    diff = []

    for key1, key2 in itertools.combinations(dict_of_mean_values.keys(), 2):
        diff.append(abs(np.subtract(np.array(dict_of_mean_values[key1]),np.array(dict_of_mean_values[key2]))))
            
    #creation of the matrix W
    D = origin_dict[1].shape[1]
    W = np.ones((D, D))
    
    for row in range(len(W)):
        for col in range(len(W[0])):
            dij = abs(row-col)
            W[row][col] = 1 - (dij/(D-1))
            
    #calculating QFD
    QFD = []
    for count,d in enumerate(diff):
        QFD.append(math.sqrt(np.dot(np.dot(d,W),np.transpose(d))))
        
    tds = statistics.mean(QFD)
    
    return tds

In [6]:
print(calculate_FS1_for_each_column(cars,"origin"))

0.6268655640398301


# FS2

In [7]:
# FS2 funkcija

def calculate_FS2_for_each_column(df, group_label):
    
    col = df.columns
    colu = []
    for c in col:
        if (isfloat(df[c][0])):
            colu.append(c)
            
    for c in colu:
        df[c] = df[c].astype(float)
    
    string_columns_float = list(df.select_dtypes(exclude=['object']).columns)
    string_columns_object = list(df.select_dtypes(include=['object']).columns)
    
    # print(f"Group label is: {group_label}")
    # print(f"Group label has {len(df[group_label].unique())} unique values")
    no_of_datasets = len(df[group_label].unique())
    df = df.drop(labels=string_columns_object,axis=1)
    list_of_datasets = []
    origin_dict = {}
    dict_of_mean_values = {} #key is the origin and the values are list of means for each column of a datas
    dict_of_std_values_minus = {}
    dict_of_std_values_plus = {}
    final_dict = {}

    #creating separated datasetst for each group
    for count,value in enumerate(df[group_label].unique()):
        list_of_datasets.append( df[df[group_label] == value])
        list_of_datasets[count] = list_of_datasets[count].reset_index()
    
    
    #drop origin column for each group and save it as a key in dictionary where 
    #the value of the key will be the corresponding dataset
    
    for dataset in list_of_datasets:
        
        group = int(dataset[group_label].unique()[0])
        
        origin_dict[group] = dataset.drop(labels=[group_label],axis=1)
        
        columns = origin_dict[group].columns
        scaler = MinMaxScaler()
        scaler.fit(origin_dict[group])
        origin_dict[group] = scaler.transform(origin_dict[group])
        origin_dict[group] = pd.DataFrame(origin_dict[group], columns = columns)
        origin_dict[group] = origin_dict[group].drop(labels=["index"] ,axis=1)
        dict_of_std_values_minus[group] = [origin_dict[group][c].mean() - origin_dict[group][c].std() for c in origin_dict[group].columns]
        dict_of_mean_values[group] = [origin_dict[group][c].mean() for c in origin_dict[group].columns]
        dict_of_std_values_plus[group] = [origin_dict[group][c].mean() + origin_dict[group][c].std() for c in origin_dict[group].columns]
        matrix = np.array([dict_of_std_values_minus[group], dict_of_mean_values[group], dict_of_std_values_plus[group]])
        final_dict[group] = matrix
        
    #calculation of diff and QDS between each combination of the groups.
    diff = []

    for key1, key2 in itertools.combinations(final_dict.keys(), 2):
        diff.append(abs(np.subtract(np.array(final_dict[key1]),np.array(final_dict[key2]))))
            
    #creation of the matrix W
    D = origin_dict[1].shape[1]
    W = np.ones((D, D))
        
    for row in range(len(W)):
        for col in range(len(W[0])):
            dij = abs(row-col)
            W[row][col] = 1 - (dij/(D-1))
            
    # print(diff)
                        
    #calculating QFD
    QFD = []
    for count,d in enumerate(diff):
        # print(d.shape)
        # print(W.shape)
        QFD.append(np.sqrt(np.dot(np.dot(d,W),np.transpose(d))))
            
    TDS = sum(QFD)/len(QFD)
    TDS = sum(TDS)/len(TDS)
    TDS = sum(TDS)/len(TDS)
    
    return TDS

In [8]:
print(calculate_FS2_for_each_column(cars,"origin"))

0.625254807610549


# FS3 

In [9]:
# FS3 funkcija

def calc_hist(numbers):
    range_0_2 = 0
    range_2_4 = 0
    range_4_6 = 0
    range_6_8 = 0
    range_8_10 = 0

    # Iterate through the numbers and count occurrences in each range
    for number in numbers:
        if 0 <= number < 0.2:
            range_0_2 += 1
        elif 0.2 <= number < 0.4:
            range_2_4 += 1
        elif 0.4 <= number < 0.6:
            range_4_6 += 1
        elif 0.6 <= number < 0.8:
            range_6_8 += 1
        elif 0.8 <= number <= 1.0:
            range_8_10 += 1

    ranges = [range_0_2, range_2_4, range_4_6, range_6_8, range_8_10]
    
    return(np.array(ranges))

def calculate_FS3_for_each_column(df, group_label):
    
    col = df.columns    
    
    colu = []
    for c in col:
        if (isfloat(df[c][0])):
            colu.append(c)
            
    for c in colu:
        df[c] = df[c].astype(float)
    
    string_columns_float = list(df.select_dtypes(exclude=['object']).columns)
    string_columns_object = list(df.select_dtypes(include=['object']).columns)
    
    # print(f"Group label is: {group_label}")
    # print(f"Group label has {len(df[group_label].unique())} unique values")
    no_of_datasets = len(df[group_label].unique())
    df = df.drop(labels=string_columns_object,axis=1)
    list_of_datasets = []
    origin_dict = {}
    final_dict = {}

    #creating separated datasetst for each group
    for count,value in enumerate(df[group_label].unique()):
        list_of_datasets.append( df[df[group_label] == value])
        list_of_datasets[count] = list_of_datasets[count].reset_index()
    
    
    #drop origin column for each group and save it as a key in dictionary where 
    #the value of the key will be the corresponding dataset
    
    for dataset in list_of_datasets:
        
        group = int(dataset[group_label].unique()[0])
        
        origin_dict[group] = dataset.drop(labels=[group_label],axis=1)
        
        columns = origin_dict[group].columns
        scaler = MinMaxScaler()
        scaler.fit(origin_dict[group])
        origin_dict[group] = scaler.transform(origin_dict[group])
        origin_dict[group] = pd.DataFrame(origin_dict[group], columns = columns)
        origin_dict[group] = origin_dict[group].drop(labels=["index"] ,axis=1)

        final_dict[group] = [(calc_hist(origin_dict[group][c]) / origin_dict[group][c].shape[0]) for c in origin_dict[group].columns]
        
    #calculation of diff and QDS between each combination of the groups.
    diff = []

    for key1, key2 in itertools.combinations(final_dict.keys(), 2):
        diff.append(abs(np.subtract(np.array(final_dict[key1]),np.array(final_dict[key2]))))
            
    # print(diff)
    
    #creation of the matrix W
    D = len(final_dict[1])
    W = np.ones((D, D))
        
    for row in range(len(W)):
        for col in range(len(W[0])):
            dij = abs(row-col)
            W[row][col] = 1 - (dij/(D-1))
                        
    #calculating QFD
    QFD = []
    for count,d in enumerate(diff):
        # print(W.shape)
        # print(d.shape)
        # QFD.append(np.sqrt(np.dot(np.dot(d,W),np.transpose(d))))
        QFD.append(np.sqrt(np.dot(np.dot(np.transpose(d),W),d)))
        
    TDS = sum(QFD)/len(QFD)
    TDS = sum(TDS)/len(TDS)
    TDS = sum(TDS)/len(TDS)
    
    return TDS

In [10]:
print(calculate_FS3_for_each_column(cars,"origin"))

0.6213322709152201


# Run the combinations for cars - fs1

In [42]:
cars = pd.read_csv("data/cars/cars.csv", index_col=0)

In [43]:
cars = cars[cars.horsepower != "?"]
df = cars.copy()

In [44]:
cars.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car_name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino


In [45]:
og_order = df.columns.tolist()

In [46]:
col_keys = [i for i in range(0, len(df.columns.tolist()))]

In [47]:
tds_list = []
permutations_list = list(permutations(col_keys))
permutations_list = [list(tup) for tup in permutations_list]

In [48]:
def run_parallel(i):

    subset = df.iloc[:, permutations_list[i]]
    return(calculate_FS1_for_each_column(subset,"origin"))

In [49]:
# calculate how much time it would take to perform the function on the entire data set

og_length = len(permutations_list)

permutations_list = permutations_list[:1000]

permutations_list_names = permutations_list.copy()

for i in range(len(permutations_list)):
    permutations_list_names[i] = [og_order[index] for index in permutations_list[i]]

start_time = time.time()
tds_list = joblib.Parallel(n_jobs=-1)(joblib.delayed(run_parallel)(i) for i in range(len(permutations_list)))
end_time = time.time()

time_takes = (end_time - start_time) * (og_length / 1000)

print(time_takes)

7872.443739624024


In [23]:
# original_function

# for i in range(len(permutations_list)):

#    subset = df.iloc[:, permutations_list[i]]
#    tds_list.append(calculate_FS1_for_each_column(subset,"origin"))

In [50]:
# tds_list = joblib.Parallel(n_jobs=-1)(joblib.delayed(run_parallel)(i) for i in range(len(permutations_list)))

In [ ]:
# the resulting dictionary along with the actual column names
result_cars_cols = {
    index: {"columns_order": permutations_list_names[index], "tds": tds_list[index]}
    for index, _ in enumerate(permutations_list_names)
}

# the resulting dictionary along with the indexes of the column names as was ordered in the original ordering
result_cars_indexes = {
    index: {"columns_order": permutations_list[index], "tds": tds_list[index]}
    for index, _ in enumerate(permutations_list)
}

In [ ]:
# result_cars_cols

In [ ]:
# result_cars_indexes

In [ ]:
file_path = 'results/cars_results.json'

with open(file_path, "w") as json_file:
    json.dump(result_dict_cols, json_file)

# Run the combinations for glass data set - fs2

In [57]:
glass = pd.read_csv("data/glass/glass.csv", index_col=0)

In [58]:
df = glass.copy()
df.drop('id', axis=1, inplace=True)

In [59]:
df.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,type_of_glass
0,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.00,1
1,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.00,1
2,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.00,1
3,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.00,1
4,1.51596,12.79,3.61,1.62,72.97,0.64,8.07,0.0,0.26,1


In [60]:
og_order = df.columns.tolist()

In [61]:
col_keys = [i for i in range(0, len(df.columns.tolist()))]

In [62]:
tds_list = []
permutations_list = list(permutations(col_keys))
permutations_list = [list(tup) for tup in permutations_list]

In [63]:
def run_parallel_2(i):

    subset = df.iloc[:, permutations_list[i]]
    return(calculate_FS2_for_each_column(subset,"type_of_glass"))

In [64]:
# calculate how much time it would take to perform the function_2 on the entire glass data set

og_length = len(permutations_list)

permutations_list = permutations_list[:1000]

permutations_list_names = permutations_list.copy()

for i in range(len(permutations_list)):
    permutations_list_names[i] = [og_order[index] for index in permutations_list[i]]

start_time = time.time()
tds_list = joblib.Parallel(n_jobs=-1)(joblib.delayed(run_parallel_2)(i) for i in range(len(permutations_list)))
end_time = time.time()

time_takes = (end_time - start_time) * (og_length / 1000)

print(time_takes)

211616.51675262454


In [ ]:
# original_code

# for i in range(len(permutations_list)):
#     cols_ordering = permutations_list[i]
#     
#     subset = df.iloc[:, cols_ordering]
#     tds_list.append(calculate_FS2_for_each_column(subset,"type_of_glass"))

In [ ]:
# the resulting dictionary along with the actual column names
result_glass_cols = {
    index: {"columns_order": permutations_list_names[index], "tds": tds_list[index]}
    for index, _ in enumerate(permutations_list_names)
}

# the resulting dictionary along with the indexes of the column names as was ordered in the original ordering
result_glass_indexes = {
    index: {"columns_order": permutations_list[index], "tds": tds_list[index]}
    for index, _ in enumerate(permutations_list)
}

In [ ]:
# result_glass_cols

In [ ]:
# result_glass_indexes

In [ ]:
file_path = 'results/glass_results.json'

with open(file_path, "w") as json_file:
    json.dump(result_dict_cols, json_file)

# Run the combinations for seeds data set - fs3

In [75]:
seeds = pd.read_csv("data/seeds/seeds.csv", index_col=0)

In [76]:
df = seeds.copy()

In [77]:
df.head()

,area,perimeter,compactness,length_kernel,width_kernel,asymmetry_coeff,length_groove,class
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


In [78]:
og_order = df.columns.tolist()

In [79]:
og_order

['area',
 'perimeter',
 'compactness',
 'length_kernel',
 'width_kernel',
 'asymmetry_coeff',
 'length_groove',
 'class']

In [80]:
col_keys = [i for i in range(0, len(df.columns.tolist()))]
col_keys = col_keys[:-1]

In [81]:
tds_list = []
permutations_list = list(permutations(col_keys))
permutations_list = [list(tup) for tup in permutations_list]

permutations_list_names = permutations_list.copy()

for i in range(len(permutations_list)):
    permutations_list_names[i] = [og_order[index] for index in permutations_list[i]]
    permutations_list_names[i].append('class')

In [82]:
for i in range(len(permutations_list)):
    permutations_list[i].append(7)

In [83]:
def run_parallel_3(i):

    subset = df.iloc[:, permutations_list[i]]
    return(calculate_FS3_for_each_column(subset,"class"))

In [84]:
"""# calculate how much time it would take to perform the function_2 on the entire glass data set

og_length = len(permutations_list)

permutations_list = permutations_list[:1000]

permutations_list_names = permutations_list.copy()

for i in range(len(permutations_list)):
    permutations_list_names[i] = [og_order[index] for index in permutations_list[i]]

start_time = time.time()
tds_list = joblib.Parallel(n_jobs=-1)(joblib.delayed(run_parallel_3)(i) for i in range(len(permutations_list)))
end_time = time.time()

time_takes = (end_time - start_time) * (og_length / 1000)

print(time_takes)"""

'# calculate how much time it would take to perform the function_2 on the entire glass data set\n\nog_length = len(permutations_list)\n\npermutations_list = permutations_list[:1000]\n\npermutations_list_names = permutations_list.copy()\n\nfor i in range(len(permutations_list)):\n    permutations_list_names[i] = [og_order[index] for index in permutations_list[i]]\n\nstart_time = time.time()\ntds_list = joblib.Parallel(n_jobs=-1)(joblib.delayed(run_parallel_3)(i) for i in range(len(permutations_list)))\nend_time = time.time()\n\ntime_takes = (end_time - start_time) * (og_length / 1000)\n\nprint(time_takes)'

In [85]:
# original_code

for cols_ordering in permutations_list:
    subset = df.iloc[:, cols_ordering]
    tds_list.append(calculate_FS3_for_each_column(subset,"class"))

In [94]:
# the resulting dictionary along with the actual column names
result_seeds_cols = {
    index: {"columns_order": permutations_list_names[index], "tds": tds_list[index]}
    for index, _ in enumerate(permutations_list_names)
}

# the resulting dictionary along with the indexes of the column names as was ordered in the original ordering
result_seeds_indexes = {
    index: {"columns_order": permutations_list[index], "tds": tds_list[index]}
    for index, _ in enumerate(permutations_list)
}

In [95]:
# result_seeds_cols

In [96]:
len(result_seeds_cols)

5040

In [97]:
len(result_seeds_indexes)

5040

In [98]:
file_path = 'results/seeds/result_seeds_cols.json'

with open(file_path, "w") as json_file:
    json.dump(result_seeds_cols, json_file)

In [99]:
file_path = 'results/seeds/result_seeds_indexes.json'

with open(file_path, "w") as json_file:
    json.dump(result_seeds_indexes, json_file)

# Getting the columns order of the best and the worst

In [100]:
result_seeds_indexes

{0: {'columns_order': [0, 1, 2, 3, 4, 5, 6], 'tds': 0.4182978566134955},
 1: {'columns_order': [0, 1, 2, 3, 4, 6, 5], 'tds': 0.41969333567997175},
 2: {'columns_order': [0, 1, 2, 3, 5, 4, 6], 'tds': 0.4170259097019192},
 3: {'columns_order': [0, 1, 2, 3, 5, 6, 4], 'tds': 0.416263107054616},
 4: {'columns_order': [0, 1, 2, 3, 6, 4, 5], 'tds': 0.41931768881458104},
 5: {'columns_order': [0, 1, 2, 3, 6, 5, 4], 'tds': 0.41713543325304114},
 6: {'columns_order': [0, 1, 2, 4, 3, 5, 6], 'tds': 0.41816406127804406},
 7: {'columns_order': [0, 1, 2, 4, 3, 6, 5], 'tds': 0.4195509598634378},
 8: {'columns_order': [0, 1, 2, 4, 5, 3, 6], 'tds': 0.42095108502341405},
 9: {'columns_order': [0, 1, 2, 4, 5, 6, 3], 'tds': 0.42822201850427594},
 10: {'columns_order': [0, 1, 2, 4, 6, 3, 5], 'tds': 0.4229849318448558},
 11: {'columns_order': [0, 1, 2, 4, 6, 5, 3], 'tds': 0.4288696643303284},
 12: {'columns_order': [0, 1, 2, 5, 3, 4, 6], 'tds': 0.41656645922089297},
 13: {'columns_order': [0, 1, 2, 5, 3, 6, 

In [104]:
max_key = max(result_seeds_indexes, key=lambda k: result_seeds_indexes[k]['tds'])
min_key = min(result_seeds_indexes, key=lambda k: result_seeds_indexes[k]['tds'])

print("Key with maximum tds value:", max_key)
print("Key with minimum tds value:", min_key)

Key with maximum tds value: 808
Key with minimum tds value: 3120


### The best ordering

In [105]:
result_seeds_indexes[max_key]

{'columns_order': [1, 0, 5, 4, 6, 2, 3], 'tds': 0.4345727485543593}

In [134]:
best_ordering = result_seeds_indexes[max_key]['columns_order']

In [124]:
df['class'].unique()

array([1, 2, 3], dtype=int64)

In [125]:
seeds_1 = df[df['class'] == 1].copy()

In [126]:
seeds_2 = df[df['class'] == 2].copy()

In [127]:
seeds_3 = df[df['class'] == 3].copy()

In [128]:
seeds_1.drop('class', axis=1, inplace=True)
seeds_2.drop('class', axis=1, inplace=True)
seeds_3.drop('class', axis=1, inplace=True)

In [143]:
seeds_1_best = seeds_1.iloc[:, best_ordering].copy()
seeds_2_best = seeds_2.iloc[:, best_ordering].copy()
seeds_3_best = seeds_3.iloc[:, best_ordering].copy()

In [106]:
result_seeds_indexes[min_key]

{'columns_order': [4, 2, 0, 1, 3, 5, 6], 'tds': 0.401919672665923}

In [138]:
worst_ordering = result_seeds_indexes[min_key]['columns_order']

In [140]:
seeds_1_worst = seeds_1.iloc[:, worst_ordering].copy()
seeds_2_worst = seeds_2.iloc[:, worst_ordering].copy()
seeds_3_worst = seeds_3.iloc[:, worst_ordering].copy()

In [146]:
seeds_1_worst.to_csv('results/seeds/orderings-data-sets/seeds_1_worst.csv')
seeds_2_worst.to_csv('results/seeds/orderings-data-sets/seeds_2_worst.csv')
seeds_3_worst.to_csv('results/seeds/orderings-data-sets/seeds_3_worst.csv')
seeds_1_best.to_csv('results/seeds/orderings-data-sets/seeds_1_best.csv')
seeds_2_best.to_csv('results/seeds/orderings-data-sets/seeds_2_best.csv')
seeds_3_best.to_csv('results/seeds/orderings-data-sets/seeds_3_best.csv')